In [1]:
import json, requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

headers = {
    'x-rapidapi-key': "fe64c2341amsh4943f70e7aa3c5cp14dd05jsnbc6fcc0f9c7b",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }


In [2]:
# manually set variables for our user
account_id = 12345
cash_amount = 1000.00
investment_account = {
    "GME": 20,
    "TSLA": 30,
    "TR": 300,
    "SPY": 75,
    "AMD": 10,
    "GOOG": 0.5,
    "AAPL": 30,
    "FB": 10,
    "NOK": 200,
    "AMC": 50,
    
}

In [3]:
# GENERATES ACCOUNT HISTORY FOR SQL
account_history_sql = []
instruments_sql = []

instruments_history = {}
three_months_ago = datetime.now().date() - relativedelta(months=3)
yesterday = datetime.now().date() - timedelta(days=1)
dates = [date.to_pydatetime().date() for date in pd.date_range(start=three_months_ago, end=yesterday)]

# kind of lazy, but need today's instrument price for instrument data
today_price = {}

for symbol in investment_account.keys():
    # go to api for historical data
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    querystring = {"symbol": symbol,"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    prices = json.loads(response.text)["prices"]

    for price in prices:
        if datetime.fromtimestamp(price["date"]).date() in dates and "adjclose" in price:
            today_price[symbol] = price["adjclose"]
            if datetime.fromtimestamp(price["date"]).date() not in instruments_history:
                instruments_history[datetime.fromtimestamp(price["date"]).date()] = {symbol: price["adjclose"] * investment_account[symbol]}
            else:
                instruments_history[datetime.fromtimestamp(price["date"]).date()][symbol] = price["adjclose"] * investment_account[symbol]
                        
# generate sql statements for account history
for date in instruments_history:
    account_history_sql.append("INSERT INTO account_history VALUES({0}, '{1}', {2});".format(account_id, str(date), round(sum(instruments_history[date].values()) + cash_amount, 2)))

In [4]:
# GENERATES INSTRUMENT DATA FOR SQL
for symbol in investment_account.keys():
    # go to api for instrument summaries
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary"
    querystring = {"symbol": symbol,"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    summary = json.loads(response.text)["quoteType"]
    
    # generate sql statements for instrument summaries
    if summary["quoteType"] == "EQUITY":
        instruments_sql.append("INSERT INTO instruments VALUES('{0}', '{1}', '{2}', {3});".format(symbol, summary["longName"], "Stock", today_price[symbol]))
    else:
        instruments_sql.append("INSERT INTO instruments VALUES('{0}', '{1}', '{2}', {3});".format(symbol, summary["longName"], summary["quoteType"].lower().title(), today_price[symbol]))   

In [19]:
other_sql = []
other_sql.append("INSERT INTO users VALUES(1001, 'password1');")
other_sql.append("INSERT INTO users VALUES(1002, 'password2');")

other_sql.append("INSERT INTO accounts VALUES(12345, 1001, 'A investment acc', 'Investment', {0});".format(round(sum(instruments_history[list(instruments_history)[0]].values()) + cash_amount, 2)))
other_sql.append("INSERT INTO accounts VALUES(67890, 1001, 'A banking acc', 'Banking', 5000.45);")
other_sql.append("INSERT INTO accounts VALUES(13579, 1001, 'A banking acc 2', 'Banking', 75.90);")

other_sql.append("INSERT INTO banking_acc_info VALUES(67890, 'Checking', 0.05);")
other_sql.append("INSERT INTO banking_acc_info VALUES(13579, 'Savings', 0.07);")

other_sql.append("INSERT INTO investment_acc_info VALUES(12345, 'RRSP', 1000);")

other_sql.append("INSERT INTO account_transactions VALUES(13579, 67890, '2021-08-31', 'shopping', 'kids bday', -200.00);")
other_sql.append("INSERT INTO account_transactions VALUES(24680, 67890, '2021-08-30', 'deposit', 'payday', 800.00);")

investments_sql = []
for symbol in investment_account:
    prices = []
    for date in instruments_history.keys():
        prices.append(instruments_history[date][symbol])
    investments_sql.append("INSERT INTO investments VALUES({0}, '{1}', {2}, {3});".format(account_id, symbol, investment_account[symbol], round(sum(prices)/len(prices), 2)))



In [24]:
file = open("insertData.sql", "a")
for statement in other_sql:
    file.write(statement + "\n")
for statement in instruments_sql:
    file.write(statement + "\n")
for statement in account_history_sql:
    file.write(statement + "\n")
for statement in investments_sql:
    file.write(statement + "\n")
file.close()